In [12]:
from __future__ import print_function, division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import seaborn as sns
import datetime
sns.set()

#source: MTA files from http://web.mta.info/developers/turnstile.html

def get_data(week_nums):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    dfs = []
    for week_num in week_nums:
        file_url = url.format(week_num)
        dfs.append(pd.read_csv(file_url))
    return pd.concat(dfs) # handles duplicate headers
        
week_nums = [180407, 180414, 180421, 180428, 180505, 180512, 
             180519, 180526, 180602, 180609, 180616, 180623, 180630]


In [2]:
# Pull data in

turnstiles_df = get_data(week_nums)
turnstiles_df.head()
                 

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/31/2018,00:00:00,REGULAR,6566463,2224050
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/31/2018,04:00:00,REGULAR,6566470,2224053
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/31/2018,08:00:00,REGULAR,6566470,2224053
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/31/2018,12:00:00,REGULAR,6566470,2224055
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,03/31/2018,16:00:00,REGULAR,6566470,2224055


In [13]:
turnstiles_df.to_csv('merged.csv', index = False)

In [14]:
turnstiles_df = pd.read_csv('merged.csv')

In [15]:
# Filter for just stations we're interested in

mask = ((turnstiles_df['STATION'] == 'GRD CNTRL-42 ST') | (turnstiles_df['STATION'] == '42 ST-PORT AUTH') \
        |  (turnstiles_df['STATION'] == 'TIMES SQ-42 ST') | (turnstiles_df['STATION'] == '42 ST-BRYANT PK'))

turnstiles_df = turnstiles_df[mask]

In [16]:
# Strip the extra spaces in the column names

turnstiles_df = turnstiles_df.rename(columns=lambda x: x.strip()) #strip spaces
#or: turnstiles_df.columns = [column.strip() for column in turnstiles_df.columns]

In [17]:
#Create a new column that stores the date and time as a single pandas datetime object

turnstiles_df['DateTime'] = turnstiles_df['DATE'] + turnstiles_df['TIME']
turnstiles_df['DateTime'] = pd.to_datetime(turnstiles_df['DateTime'], format='%m/%d/%Y%H:%M:%S')

ValueError: time data '2018-03-31 00:00:0000:00:00' does not match format '%m/%d/%Y%H:%M:%S' (match)

In [18]:
# Create columns for week number and day of week

turnstiles_df['DATE'] = pd.to_datetime(turnstiles_df['DATE'])
turnstiles_df['weeknum'] = turnstiles_df['DATE'].dt.week
turnstiles_df['daynum'] = turnstiles_df['DATE'].dt.dayofweek

In [19]:
# Calculate actual value of passengers in each time window

turnstiles_df = turnstiles_df.sort_values(by=['C/A','UNIT','SCP', 'STATION','DateTime'])
turnstiles_df['Daily_Entries'] = turnstiles_df['ENTRIES'] - turnstiles_df['ENTRIES'].shift(1)
turnstiles_df['Daily_Exits'] = turnstiles_df['EXITS'] - turnstiles_df['EXITS'].shift(1)

In [20]:
# Get rid of negative values

turnstiles_df['Daily_Entries_abs'] = turnstiles_df['Daily_Entries'].abs()
turnstiles_df['Daily_Exits_abs'] = turnstiles_df['Daily_Exits'].abs()

In [21]:
stations

NameError: name 'stations' is not defined

In [22]:
from bokeh.transform import jitter


output_file("categorical_scatter_jitter.html")

stations = list(turnstiles_df['STATION'].unique())

source = ColumnDataSource(turnstiles_df)

p = figure(plot_width=1000, plot_height=600, y_range=stations, 
           title="Time Period Entries by Station")

p.circle(x='Daily_Entries_abs', y=jitter('STATION', width=0.6, range=p.y_range),  source=source, alpha=0.6)

p.title.text_font_size='20pt'
p.title.text_font_style='bold'

p.yaxis.axis_label='Station'
p.yaxis.axis_label_standoff=20
p.yaxis.axis_label_text_font_size='15pt'
p.yaxis.major_label_text_font_size='13pt'

p.xaxis.axis_label='Number of Commuters'
p.xaxis.axis_label_standoff=20
p.xaxis.axis_label_text_font_size='15pt'
p.xaxis[0].formatter = NumeralTickFormatter(format="0.0a")
p.xaxis.major_label_text_font_size='13pt'

show(p)

NameError: name 'output_file' is not defined

In [ ]:
#remove large numbers

def replace(group):
    upper = 100000
    outliers = group > upper
    group[outliers] = np.nan  
    return group

turnstiles_df['Daily_Entries_abs'] = turnstiles_df['Daily_Entries_abs'].transform(replace)
turnstiles_df['Daily_Exits_abs'] = turnstiles_df['Daily_Exits_abs'].transform(replace)

In [ ]:
# Replace any na's with data point before

turnstiles_df['Daily_Entries_abs'].fillna(method='ffill', inplace=True)
turnstiles_df['Daily_Exits_abs'].fillna(method='ffill', inplace=True)

turnstiles_df['Daily_Entries_abs'].fillna(method='bfill', inplace=True)
turnstiles_df['Daily_Exits_abs'].fillna(method='bfill', inplace=True)

In [ ]:
from bokeh.transform import jitter


output_file("categorical_scatter_jitter.html")

stations = list(turnstiles_df['STATION'].unique())

source = ColumnDataSource(turnstiles_df)

p = figure(plot_width=1000, plot_height=600, y_range=stations, 
           title="Time Period Entries by Station")

p.circle(x='Daily_Entries_abs', y=jitter('STATION', width=0.6, range=p.y_range),  source=source, alpha=0.6)

p.title.text_font_size='20pt'
p.title.text_font_style='bold'

p.yaxis.axis_label='Station'
p.yaxis.axis_label_standoff=20
p.yaxis.axis_label_text_font_size='15pt'
p.yaxis.major_label_text_font_size='13pt'

p.xaxis.axis_label='Number of Commuters'
p.xaxis.axis_label_standoff=20
p.xaxis.axis_label_text_font_size='15pt'
p.xaxis[0].formatter = NumeralTickFormatter(format="0.0a")
p.xaxis.major_label_text_font_size='13pt'

show(p)

In [ ]:
# Create combined entries and exits column

turnstiles_df['Combined_Traffic'] = turnstiles_df['Daily_Entries_abs'] + turnstiles_df['Daily_Exits_abs']

turnstiles_df['Combined_Traffic'] = turnstiles_df['Combined_Traffic'].astype(int)

In [ ]:
# Group by station and datetime

timesquare_stations = \
            (turnstiles_df.groupby(['STATION','DATE','weeknum', 'daynum','TIME'])[['Combined_Traffic']]
             .sum()
             .reset_index())   #retains row label

In [ ]:
# Remove any large numbers that don't make sense

timesquare_stations['Combined_Traffic'] = timesquare_stations['Combined_Traffic'].transform(replace)
timesquare_stations['Combined_Traffic'].fillna(method='ffill', inplace=True)
timesquare_stations['Combined_Traffic'].fillna(method='bfill', inplace=True)

timesquare_stations['Combined_Traffic'] = timesquare_stations['Combined_Traffic'].astype(int)

In [ ]:
# Create table with total traffic for each station

top_stations = \
            (timesquare_stations.groupby(['STATION'])[['Combined_Traffic']]
             .sum()
             .reset_index())   #retains row label

In [ ]:
# Create graph to show which stations are the most trafficked

from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral6
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.models import NumeralTickFormatter

output_file("topstations.html")

stations = top_stations['STATION'].tolist()

p = figure(x_range=stations, plot_height=600, plot_width=1000, title="Traffic Volume at 42nd St Stations",
           toolbar_location=None, tools="")


p.vbar(x=stations, top=top_stations['Combined_Traffic'], width=0.5, fill_color='green', fill_alpha=0.5)

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.title.text_color='olive'
p.title.text_font_size='20pt'
p.title.text_font_style='bold'

p.xaxis.axis_label='Station'
p.xaxis.axis_label_text_color='olive'
p.xaxis.axis_label_standoff=20
p.xaxis.axis_label_text_font_size='15pt'
p.xaxis.major_label_text_font_size='13pt'

p.yaxis.axis_label='Number of Commuters'
p.yaxis.axis_label_text_color='olive'
p.yaxis.axis_label_standoff=20
p.yaxis.axis_label_text_font_size='15pt'
p.yaxis[0].formatter = NumeralTickFormatter(format="0.0a")
p.yaxis.major_label_text_font_size='13pt'

show(p)


In [ ]:
# Create table with just grand central, the most trafficked station

grandcentral = timesquare_stations[timesquare_stations['STATION'] == 'GRD CNTRL-42 ST']

In [ ]:
# Create table by day of week for grand central

grandcentral_by_day = \
            (grandcentral.groupby(['STATION','daynum'])[['Combined_Traffic']]
             .sum()
             .reset_index()) 

In [ ]:
# Make graph to show which days of the week are most used

from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral6
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.models import NumeralTickFormatter

output_file("grandcentral_byday.html")

days = grandcentral_by_day['daynum'].tolist()
weekdays = dict([(0,'Monday'), (1,'Tuesday'), (2,'Wednesday'), (3,'Thursday'), (4,'Friday'), (5,'Saturday'), (6,'Sunday')])

p = figure(plot_height=600, plot_width=1000, title="Traffic Volume at Grand Central",
           toolbar_location=None, tools="")


p.line(x=days, y=grandcentral_by_day['Combined_Traffic'], line_width=4, color='green')

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.title.text_color='olive'
p.title.text_font_size='20pt'
p.title.text_font_style='bold'

p.xaxis.axis_label='Day of Week'
p.xaxis.axis_label_text_color='olive'
p.xaxis.axis_label_standoff=20
p.xaxis.axis_label_text_font_size='15pt'
p.xaxis.major_label_text_font_size='13pt'
p.xaxis.major_label_overrides=weekdays

p.yaxis.axis_label='Number of Commuters'
p.yaxis.axis_label_text_color='olive'
p.yaxis.axis_label_standoff=20
p.yaxis.axis_label_text_font_size='15pt'
p.yaxis[0].formatter = NumeralTickFormatter(format="0.0a")
p.yaxis.major_label_text_font_size='13pt'

show(p)


#fig, ax = plt.subplots()
#for name, group in grandcentral_by_day.groupby(['weeknum']):
#    group.plot('daynum', y='Combined_Traffic', ax=ax, label=name)


In [ ]:
# Filter for only the weekdays

grandcentral_weekdays = grandcentral[grandcentral['daynum'] != 5]
grandcentral_weekdays = grandcentral_weekdays[grandcentral_weekdays['daynum'] != 6]

In [ ]:
# Pull the hour from the time field

grandcentral_weekdays['TIME'] = pd.to_datetime(grandcentral_weekdays['TIME'], format='%H:%M:%S').dt.hour

In [ ]:
# Create buckets for each hour

bins = [-1, 3, 7, 11, 15, 19, 23]

grandcentral_weekdays['TimeBucket'] = pd.cut(grandcentral_weekdays['TIME'], bins)

grandcentral_weekdays['TimeBucket'] = grandcentral_weekdays['TimeBucket'].astype(str)

In [ ]:
# Convert intervals to more usable number

timebucketindex = pd.DataFrame([['(-1, 3]',0], ['(3, 7]',1], ['(7, 11]',2], \
                    ['(11, 15]',3], ['(15, 19]',4], ['(19, 23]',5]], columns=['TimeBucket','TimeBucketIndex'])

grandcentral_weekdays = pd.merge(grandcentral_weekdays,timebucketindex,on='TimeBucket')

In [ ]:
#Create table grouped by time bucket

grandcentral_weekdays_bytime = \
            (grandcentral_weekdays.groupby(['STATION','TimeBucketIndex'])[['Combined_Traffic']]
             .sum()
             .reset_index()) 

In [ ]:
# Create graph that shows which time periods are most trafficked

from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral6
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.models import NumeralTickFormatter

output_file("grandcentral_weekday_bytime2.html")

times = grandcentral_weekdays_bytime['TimeBucketIndex'].tolist()

timeperiods = dict([(0,'8pm - 12am'), (1,'12am - 4am'), (2,'4am - 8am'), \
                    (3,'8am - 12pm'), (4,'12pm - 4pm'), (5,'4pm - 8pm')])

p = figure(plot_height=600, plot_width=1000, title="Traffic Volume at Grand Central By Time of Day",
           toolbar_location=None, tools="")


p.line(x=times, y=grandcentral_weekdays_bytime['Combined_Traffic'], line_width=4, color='green')

p.xgrid.grid_line_color = None
p.y_range.start = 0
p.title.text_color='olive'
p.title.text_font_size='20pt'
p.title.text_font_style='bold'

p.xaxis.axis_label='Time of Day'
p.xaxis.axis_label_text_color='olive'
p.xaxis.axis_label_standoff=20
p.xaxis.axis_label_text_font_size='15pt'
p.xaxis.major_label_text_font_size='13pt'
p.xaxis.major_label_overrides=timeperiods

p.yaxis.axis_label='Number of Commuters'
p.yaxis.axis_label_text_color='olive'
p.yaxis.axis_label_standoff=20
p.yaxis.axis_label_text_font_size='15pt'
p.yaxis[0].formatter = NumeralTickFormatter(format="0.0a")
p.yaxis.major_label_text_font_size='13pt'

show(p)

In [ ]:
# Filter for turnstiles within grand central

grandcentral_turnstile = turnstiles_df[turnstiles_df['STATION'] == 'GRD CNTRL-42 ST']

In [ ]:
# Group up to the control area

grandcentral_turnstile2 = \
            (grandcentral_turnstile.groupby(['STATION','C/A','daynum','TIME'])[['Combined_Traffic']]
             .sum()
             .reset_index())   #retains row label

In [ ]:
# Filter for weekdays

grandcentral_turnstile_weekdays = grandcentral_turnstile2[grandcentral_turnstile2['daynum'] != 5]
grandcentral_turnstile_weekdays = grandcentral_turnstile_weekdays[grandcentral_turnstile_weekdays['daynum'] != 6]

In [ ]:
# Create table for control area by time

grandcentral_turnstile_wd_bytime = \
            (grandcentral_turnstile_weekdays.groupby(['STATION','C/A','TIME'])[['Combined_Traffic']]
             .sum()
             .reset_index())   #retains row label

In [ ]:
# Create time buckets

grandcentral_turnstile_wd_bytime['TIME'] = pd.to_datetime(grandcentral_turnstile_wd_bytime['TIME'], format='%H:%M:%S').dt.hour

grandcentral_turnstile_wd_bytime['TimeBucket'] = pd.cut(grandcentral_turnstile_wd_bytime['TIME'], bins)
grandcentral_turnstile_wd_bytime['TimeBucket'] = grandcentral_turnstile_wd_bytime['TimeBucket'].astype(str)

grandcentral_turnstile_wd_bytime = pd.merge(grandcentral_turnstile_wd_bytime,timebucketindex,on='TimeBucket')

In [ ]:
# Create readable labels for each time bucket

test = grandcentral_turnstile_wd_bytime.drop(columns=['STATION', 'TIME', 'TimeBucket'])

test['TimeBucketIndex'] = test['TimeBucketIndex'].astype(str)

timeperiods2 = dict([('0','8pm - 12am'), ('1','12am - 4am'), ('2','4am - 8am'), \
                    ('3','8am - 12pm'), ('4','12pm - 4pm'), ('5','4pm - 8pm')])

test = test.replace({'TimeBucketIndex': timeperiods2})

In [ ]:
# Create pivot table to be used in heat map creation

df = test.pivot_table(index='C/A', columns='TimeBucketIndex', values='Combined_Traffic')

cols = df.columns.tolist()
cols = [cols[5]] + [cols[0]] + [cols[2]] + [cols[4]] + [cols[1]] + [cols[3]]
df = df[cols]

In [ ]:
# Create heat map of control area by time

from bokeh.models import BasicTicker, ColorBar, ColumnDataSource, LinearColorMapper, PrintfTickFormatter
from bokeh.transform import transform


source = ColumnDataSource(test)

colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
mapper = LinearColorMapper(palette=colors, low=test.Combined_Traffic.min(), high=test.Combined_Traffic.max())

timeperiods = dict([('0','12am - 4am'), ('1','4am - 8am'), ('2','8am - 12pm'), \
                    ('3','12pm - 4pm'), ('4','4pm - 8pm'), ('5','8pm - 12am')])

p = figure(plot_width=1000, plot_height=600, title="Traffic Volume at Grand Central By Time of Day By Control Area",
           x_range=list(df.index), y_range=list(df.columns),
           toolbar_location=None, tools="", x_axis_location="above")

p.rect(x="C/A", y="TimeBucketIndex", width=1, height=1, source=source,
       line_color=None, fill_color=transform('Combined_Traffic', mapper))

color_bar = ColorBar(color_mapper=mapper, location=(0, 0),
                     ticker=BasicTicker(desired_num_ticks=len(colors)),
                     label_standoff=12,
                     formatter=NumeralTickFormatter(format="0.0a"))

p.add_layout(color_bar, 'right')

p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "13pt"
p.axis.major_label_standoff = 0
#p.xaxis.major_label_orientation = 1.0
p.yaxis.major_label_overrides = timeperiods2
p.yaxis.axis_label='Time of Day'
p.yaxis.axis_label_text_font_size='15pt'
p.xaxis.axis_label='Control Unit'
p.xaxis.axis_label_text_font_size='15pt'
p.title.text_font_size='20pt'

show(p)